In [7]:
import sys
sys.path.append('ReinforcementLearningAtoZ')
#jupyter에서는 폴더경로를 추가할때는 처음에는 /를 안붙여도 된다.

['c:\\users\\sy\\anaconda3\\envs\\rlcode\\python36.zip',
 'c:\\users\\sy\\anaconda3\\envs\\rlcode\\DLLs',
 'c:\\users\\sy\\anaconda3\\envs\\rlcode\\lib',
 'c:\\users\\sy\\anaconda3\\envs\\rlcode',
 '',
 'c:\\users\\sy\\anaconda3\\envs\\rlcode\\lib\\site-packages',
 'c:\\users\\sy\\gym\\baselines',
 'c:\\users\\sy\\anaconda3\\envs\\rlcode\\lib\\site-packages\\win32',
 'c:\\users\\sy\\anaconda3\\envs\\rlcode\\lib\\site-packages\\win32\\lib',
 'c:\\users\\sy\\anaconda3\\envs\\rlcode\\lib\\site-packages\\Pythonwin',
 'c:\\users\\sy\\anaconda3\\envs\\rlcode\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\sy\\.ipython',
 'ReinforcementLearningAtoZ']

In [8]:
import gym
import torch
import matplotlib.pyplot as plt

from src.part3.MLP import MultiLayerPerceptron as MLP
from src.part4.PolicyGradient import REINFORCE
from src.common.train_utils import EMAMeter, to_tensor

In [10]:
env = gym.make('CartPole-v1')
s_dim = env.observation_space.shape[0]
a_dim = env.action_space.n

## REINFORCE 알고리즘으로, 'CartPole-V1'을 정복하기.

이번 실습에서는 `REINFORCE` 알고리즘으로 `CartPole-V1` MDP를 정복해볼까요?
`REINFORCE`의 의사코드는 다음과 같습니다.

<img src="./images/REINFORCE.png" width="60%" height="40%" title="REINFORCE" alt="REINFORCE"></img>

`REINFORCE` 알고리즘을 파이썬으로는 어떻게 구현할까요?
```python
import torch.nn as nn
from torch.distributions.categorical import Categorical

class REINFORCE(nn.Module):

    def __init__(self,
                 policy: nn.Module,
                 gamma: float = 1.0,
                 lr: float = 0.0002):
        super(REINFORCE, self).__init__()
        self.policy = policy  # make sure that 'policy' returns logits!
        self.gamma = gamma
        self.opt = torch.optim.Adam(params=self.policy.parameters(),
                                    lr=lr)

        self._eps = 1e-25

    def get_action(self, state):
        with torch.no_grad():
            logits = self.policy(state)
            dist = Categorical(logits=logits)
            a = dist.sample()  # sample action from softmax policy
        return a
```

### Logit? Catergorical?

Catergorical distribution (다항분포) 는 이항분포의 확장판입니다. <br>

>이항분포는 사건의 종류가 2개일 상황을 (예를 들어 한번 동전을 던져서 앞/뒤가 나올 확률) 모델링 할 때 자주 쓰이게 됩니다.
이항 분포의 매개변수는 $p$ 로 하나의 사건이 나올 확률을 표현합니다. 이때 $0 \leq p \leq 1$ 를 따르게 됩니다.

그렇다면 사건의 종류가 2개보다 많은 경우는 어떻게 할 수 있을까요? 그런 경우에 다항 분포를 사용해볼 수 있습니다.

> 사건의 종류가 $n$ 개 일때, 다항분포의 매개변수들 $(p_1, ..., p_n)$ 이 되며, $p_i$ 는 각 사건이 일어날 확률을 표현합니다.
> 따라서 $p_i$ 는 모두 0보다 크거나 같고, $\sum_{i=1}^{n} p_i =1$ 을 만족시켜야 합니다.

자 그러면, 이제 신경망 `self.policy(state)` 출력이 위의 조건을 만족시키게 만들 수 있을까요? 다양한 방법이 존재하지만, 가장 일반적인 선택은
softmax라는 연산자를 활용하는 것입니다.

$$\sigma(z)_i = \frac{e^{z_i}}{\sum_{j=1}^{n}e^{z_j}} \forall i=1,...,n$$

임의의 값들의 집합 $\{{z_i}\}_{i=1}^{n}$ 에 위에서 정의한 softmax 연산자를 가해주면, 모든 값들은 0 이상의 값으로 바뀌고 값들의 합은 1.0 이 되게 됩니다.
이때, $z_i$ 를 일반적으로 logit 이라고 부릅니다.